# EXTRACTION
### webscraps inec database

In [58]:
def inec_webscrap(state_opt,lga_opt,ward_opt,polling_opt):
    #driver for Chrome
    from selenium.webdriver import Chrome
    #selector for dropdown
    from selenium.webdriver.support.ui import Select
    #waits until a process or page is accessible 
    from selenium.webdriver.support.ui import WebDriverWait
    #By.XPATH, By.CSS_SELECTOR etc
    from selenium.webdriver.common.by import By
    #manipulates properties of chrome driver like add fake user agent
    from selenium.webdriver.chrome.options import Options
    #certain conditions need to be met before a process begins
    from selenium.webdriver.support import expected_conditions as EC
    #this changes the useragent name which reduces reCAPTCHA verification
    from fake_useragent import UserAgent
    #allows code to sleep for a little time until process is completely done
    import time
    #to get nothing more than 5 random pages
    import random
    #this allows the random list to be sorted
    import more_itertools as mit
    #pandas cause why not
    import pandas as pd
    #needed for saving .csv
    import os
    #needed for picture download through link
    import requests

    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    options.add_argument(f'user-agent={userAgent}')

    #access to the path of your chromedriver.exe
    driver = Chrome('C:/Users/HP/Downloads/inec_scum/chromedriver.exe')
    #the homepage since you have to sign in first before navigating to the required page for sign in
    driver.get("https://cvr.inecnigeria.org/Home/start")
    #waits until your page loads completely before sign in process begins
    driver.implicitly_wait(20)

    #sign in cuse using xpath and the id of the html properties as key as well ass value for login button
    driver.find_element_by_xpath("""//*[@id="LoginEmail"]""").send_keys('')
    driver.find_element_by_xpath("""//*[@id="LoginPassword"]""").send_keys('')
    driver.find_element_by_xpath("""//*[@value="Log in"]""").click()

    #navigates to the page for inputting state, LGA,AREA id and polling unit
    driver.get("https://cvr.inecnigeria.org/VotersRegister")
    driver.implicitly_wait(6)
    states = []
    lgas = []
    ward = []
    polls = []

    select1 = Select(driver.find_element_by_id('VoterRegisterStateId'))
    #selecting by value instead of state name
    select1.select_by_visible_text(state_opt)
    state =driver.find_elements_by_id("VoterRegisterStateId")
    for sta in state:
        stat= sta.get_attribute('innerText')
        states.append(stat)
    select2 = Select(driver.find_element_by_id('VoterRegisterLocalGovernmentId'))
    select2.select_by_visible_text(lga_opt)
    lga =driver.find_elements_by_id("VoterRegisterLocalGovernmentId")
    for lg in lga:
        localgov= lg.get_attribute('innerText')
        lgas.append(localgov)
    select3 = Select(driver.find_element_by_id('VoterRegisterRegistrationAreaId'))
    select3.select_by_visible_text(ward_opt)
    area =driver.find_elements_by_id("VoterRegisterRegistrationAreaId")
    for are in area:
        areaid= are.get_attribute('innerText')
        ward.append(areaid)
    select4 = Select(driver.find_element_by_id('VoterRegisterPollingUnitId'))
    select4.select_by_visible_text(polling_opt)
    poll =driver.find_elements_by_id("VoterRegisterPollingUnitId")
    for pol in poll:
        pollid= pol.get_attribute('innerText')
        polls.append(pollid)

    #waits for page is ready then switches to the reCAPTCHA iframe
    WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,"//iframe[@title='reCAPTCHA']")))
    #clicks on the reCAPTCHA button and it either verifies without image test or it doesn't. currently at 8 out of 10 times with 8 being verification without image test
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.recaptcha-checkbox-border"))).click()


    time.sleep(5)
    #switches from reCAPTCHA iframe back to default reCAPTCHA else your code will gives error about not recognizing the next xpath
    driver.switch_to.default_content()
    #clicks on display button
    driver.find_element_by_xpath("""//*[@value="DISPLAY REGISTER"]""").click()
    driver.implicitly_wait(20)
    names = []
    loca = []
    pictures = []
    pages = []
    #required data can be found in the table with class name below that contains the name, gender, date of birth and vin for everyone in that page
    contents =driver.find_elements_by_class_name("table.table-condensed.table-borderless.condensed-rows.voter")
    for i in contents:
        name = i.get_attribute('innerText')
        names.append(name)
    #required data can be found in the table with class name below that contains the lga, polling unit, area and delim for everyone
    location =driver.find_elements_by_class_name("table.table-bordered.table-striped.afterpagebreak.bg-white")
    for s in location:
        local= s.get_attribute('innerText')
        loca.append(local)
    #to get the pictures of the registered voters under the img tag
    picture = driver.find_elements_by_css_selector("img")
    for p in picture:
        pic= p.get_attribute('src')
        pictures.append(pic)
    #this is to get the extent of the pages like "page 1 of 50"
    page = driver.find_elements_by_class_name("col-lg-12.text-right.nav-text")
    for a in page:
        pag= a.text
        pages.append(pag)

    def all_pages(web_page):
        """this function takes the next webpage https://cvr.inecnigeria.org/voters_register/index/display/page:2 as a
            parameter and returns the basic information of the registered voters in a polling unit as well as their pictures 
             and appends it to the universal list with the appopriate name
        """
        driver.get(web_page)
        contents =driver.find_elements_by_class_name("table.table-condensed.table-borderless.condensed-rows.voter")
        for i in contents:
            name = i.get_attribute('innerText')
            names.append(name)
        picture = driver.find_elements_by_css_selector("img")
        for p in picture:
            pic= p.get_attribute('src')
            pictures.append(pic)

    #after raw extract of the pages the content was split
    q =[item.split() for item in pages]
    #the extent of the page number has the below index
    page_no = q[0][5] 
    #it must be an integar or else it can't be parsed into the variable list a that has a range 
    page_num = int(page_no)
    #this list is a formatted string that returns only 5 random inec display pages based on the page_num and returns the main page if the page_num is 1,
    a = [f'https://cvr.inecnigeria.org/voters_register/index/display/page:{i:d}' for i in (mit.random_combination(range(2, page_num), r=5))] if page_num >= 7 else [f'https://cvr.inecnigeria.org/voters_register/index/display/page:{i:d}' for i in (mit.random_combination(range(2, page_num), r=page_num -2))] if 3 <= page_num <= 6 else ['https://cvr.inecnigeria.org/voters_register/index/display/page:2'] if page_num == 2 else ['https://cvr.inecnigeria.org/voters_register/index']
    #a = [f'https://cvr.inecnigeria.org/voters_register/index/display/page:{i:d}' for i in range(2, page_num +1 , 1)] 

    #this loop uses the a variable list and runs a loop for every page increment using 
    # the all_pages function that returns the basic info and pictures and appends it to the required list awaiting 
    #  awaiting cleaning and wrangling
    for o in a:
        all_pages(o)
    #ends the driver process

    driver.quit()

    #turns the area list into a pandas dataframe and changes '\n' and '\t' to ','
    #also changes the other repeated headers to nothing
    af = pd.DataFrame(loca,columns =['area'])
    af.area = af.area.apply(lambda x: x.replace('\n', '|'))
    af.area = af.area.apply(lambda x: x.replace('\t', '|'))
    af.area = af.area.apply(lambda x: x.replace('LGA:|', ''))
    af.area = af.area.apply(lambda x: x.replace('|POLLING UNIT:', ''))
    af.area = af.area.apply(lambda x: x.replace('|WARD:', ''))
    af.area = af.area.apply(lambda x: x.replace('|DELIM:', ''))
    af.area = af.area.apply(lambda x: x.replace('|RECORDS:', ''))
    af = af['area'].str.split("|",n = 6, expand = True)
    af.columns = ['LGA', 'Polling_unit', 'Ward', 'Delim','Records']
    #af = af.drop(['new1','new2'], axis=1)

    df = pd.DataFrame(names,columns =['Names'])
    df = df.replace('\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0', '')
    df.Names = df.Names.apply(lambda x: x.replace('\t\n\t', '|'))
    df.Names = df.Names.apply(lambda x: x.replace('\n\n', '|'))
    df.Names = df.Names.apply(lambda x: x.replace('VIN:', ''))
    df.Names = df.Names.apply(lambda x: x.replace('DOB-Y:', ''))
    df.Names = df.Names.apply(lambda x: x.replace('GENDER:', '|'))
    df = df['Names'].str.split("|",n=6, expand = True)
    df.columns = ['no_', 'Voters_name', 'VIN','DOB','Gender']
    df = df.drop(['no_'], axis=1)

    #concats both df and af datafrmaes together
    ed = pd.concat([df, af], axis=1)
    #since the af dataframe only has one row that corresponds to the df counterpart, fill the nan values with anything above it
    ed= ed.fillna(method='ffill')
   
    unwanted = {'https://cvr.inecnigeria.org/assets/img/myvoter.png','https://cvr.inecnigeria.org/img/logo.png'}

    pictures = [ele for ele in pictures if ele not in unwanted]

    ed['Picture_link'] = pictures

    if (os.path.exists(state_opt+".csv") == False):
        ed.to_csv(state_opt+'.csv',index=False)
    else:
        ed.to_csv(state_opt+'.csv', mode='a', index=False, header=False)
        
    first_message = "Five random pages for {polling} polling unit in the {areaing} ward of {lgaing} LGA has been successfully webscraped \n {numbe} rows were cleaned and saved in {stateing} .csv".format(polling = polling_opt,areaing =ward_opt,lgaing = lga_opt,numbe = len(ed),stateing=state_opt)
    print(first_message)
    second_message = "{} pictures of voters are currently been downloaded to your current path, \n please have data and avoid using GLO or MTN on edge browser".format(len(ed))
    print(second_message)
    print('Downloading Pictures, please be patient, this is a python code not RUST or Java')

    def download_pic(pict,name):
        url = pict
        filename = str(name)+'.jpg'
        r = requests.get(url, allow_redirects=True)
        open(filename, 'wb').write(r.content)

    for pi,na in zip(ed.Picture_link,ed.VIN):
        download_pic(pi,na)
        
        
    print('All pictures assigned to their respective VINs have successfully been downloaded, please keep up the good work') 
    return ed



In [61]:
inec_webscrap(state_opt ='RIVERS',lga_opt ='BONNY',ward_opt = 'KALAIBIAMA',polling_opt = 'VILLAGE SQUARE, OGBOKUKIRI')



Five random pages for VILLAGE SQUARE, OGBOKUKIRI polling unit in the KALAIBIAMA ward of BONNY LGA has been successfully webscraped 
 120 rows were cleaned and saved in RIVERS .csv
120 pictures of voters are currently been downloaded to your current path, 
 please have data and avoid using GLO or MTN on edge browser
All pictures assigned to their respective VINs have successfully been downloaded, please keep up the good work


,First_name,Last_name,VIN,DOB,Gender,LGA,Polling_unit,Ward,Delim,Records,Picture_link
0,1,"ABBEY, PAUL A.",90F5B1869C418471763,1987,M,BONNY,"VILLAGE SQUARE, OGBOKUKIRI",KALAIBIAMA,32-07-12-010,139,https://6nnwbexvzjtfzl5schommtcz3a0ulnlu.lambd...
1,2,"ABBEY, VICTORIA E.",FA5B3EC361532541252,1983,F,BONNY,"VILLAGE SQUARE, OGBOKUKIRI",KALAIBIAMA,32-07-12-010,139,https://6nnwbexvzjtfzl5schommtcz3a0ulnlu.lambd...
2,3,"ADIMA, MOSES K.",90F5B1869C418652661,1993,M,BONNY,"VILLAGE SQUARE, OGBOKUKIRI",KALAIBIAMA,32-07-12-010,139,https://6nnwbexvzjtfzl5schommtcz3a0ulnlu.lambd...
3,4,"Afuwajomo, Consider T.",90F5B0595C516869399,1998,F,BONNY,"VILLAGE SQUARE, OGBOKUKIRI",KALAIBIAMA,32-07-12-010,139,https://6nnwbexvzjtfzl5schommtcz3a0ulnlu.lambd...
4,5,"Alate, Wisdom S.",90F5B0595C516862809,1983,M,BONNY,"VILLAGE SQUARE, OGBOKUKIRI",KALAIBIAMA,32-07-12-010,139,https://6nnwbexvzjtfzl5schommtcz3a0ulnlu.lambd...
...,...,...,...,...,...,...,...,...,...,...,...
115,116,"TIMEYEMI, JEREMIAH B.",90F5B1869C418471035,1995,M,BONNY,"VILLAGE SQUARE, OGBOKUKIRI",KALAIBIAMA,32-07-12-010,139,https://6nnwbexvzjtfzl5schommtcz3a0ulnlu.lambd...
116,117,"TOBI, ROSE O.",90F5B1869C418464367,1961,F,BONNY,"VILLAGE SQUARE, OGBOKUKIRI",KALAIBIAMA,32-07-12-010,139,https://6nnwbexvzjtfzl5schommtcz3a0ulnlu.lambd...
117,118,"TOBIAS, GLADYS",90F5B1869C418475356,1984,M,BONNY,"VILLAGE SQUARE, OGBOKUKIRI",KALAIBIAMA,32-07-12-010,139,https://6nnwbexvzjtfzl5schommtcz3a0ulnlu.lambd...
118,119,"TOBIN, MIRIAN B.",FA5B3EC361532103147,1989,F,BONNY,"VILLAGE SQUARE, OGBOKUKIRI",KALAIBIAMA,32-07-12-010,139,https://6nnwbexvzjtfzl5schommtcz3a0ulnlu.lambd...


In [39]:
"""lagos abuga rivers imo bornu

Chinenye : LAGOS, IMO
Daniel : ABUJA, BORNU
Michael : RIVERS
"""

'lagos abuga rivers imo bornu\n\nChinenye : LAGOS, IMO\nDaniel : ABUJA, BORNU\nMichael : RIVERS\n'

In [ ]:
#logic for extracting the states and putting it in a list of lists for easy looping.

def state_listfunc():
    #Using selenium first extract list of states and assign to a list.
    state_list = [[state] for state in extracted_state_list]
    for state in state_list:
        state_to_extractfrom = state[0]
        #Using selenium extract the LGAs relating to this particular "state_to_extractfrom" assign it to the list in list form.
        LGA_list = [[LGA] for LGA in extracted_LGA_list]
        state.append(LGA_list)
        for LGA in LGA_list:
            LGA_to_extractfrom = LGA[0]
            #Using selenium extract the wards relating to this particular "LGA_to_extractfrom" assign it to the list in list form.
            ward_list = [[ward] for ward in extracted_ward_list]
            LGA.append(ward_list)
            for ward in ward_list:
                ward_to_extractfrom = LGA[0]
                #Using selenium extract the wards relating to this particular "ward_to_Extractfrom" assign it to the list in list form.
                pollingunit_list = [punit for punit in extracted_pollingunit_list]
                ward.append(pollingunit_list)
    return (state_list)

In [ ]:
#Example with "end of list" indicator to help prevent confusion from stacked brakcet
#The list of list will look like this if the code logic is executed successfully.
state_list = [['lagos',[['LGA1',[['ward1',['punit1','punit2']],['ward2',['punit1','punit2']],'end of wardlist']],
               ['LGA2',[['ward1',['punit1','punit2']],['ward2',['punit1','punit2']],'end of wardlist']],'end of LGAlist']],
              ['anambra',[['LGA1',[['ward1',['punit1','punit2']],['ward2',['punit1','punit2']],'end of wardlist']],
               ['LGA2',[['ward1',['punit1','punit2']],['ward2',['punit1','punit2']],'end of wardlist']],'end of LGAlist']],
              ['rivers',[['LGA1',[['ward1',['punit1','punit2']],['ward2',['punit1','punit2']],'end of wardlist']],
               ['LGA2',[['ward1',['punit1','punit2']],['ward2',['punit1','punit2']],'end of wardlist']],'end of LGAlist']],'end of statelist'
             ]

In [ ]:
#Example without "end of list" indicator.
#The list of list will look like this if the code logic is executed successfully.
state_list = [['lagos',[['LGA1',[['ward1',['punit1.1','punit1.2','punit1.3']],['ward2',['punit2.1','punit2.2','punit2.3']]]],
               ['LGA2',[['ward1',['punit1','punit2']],['ward2',['punit1','punit2']]]]]],
              ['anambra',[['LGA1',[['ward1',['punit1','punit2']],['ward2',['punit1','punit2']]]],
               ['LGA2',[['ward1',['punit1','punit2']],['ward2',['punit1','punit2']]]]]],
              ['rivers',[['LGA1',[['ward1',['punit1','punit2']],['ward2',['punit1','punit2']]]],
               ['LGA2',[['ward1',['punit1','punit2']],['ward2',['punit1','punit2']]]]]]
             ]

In [ ]:
#looping through the list to get the arguments for the extraction function for a certain specified state.
state = 'lagos' #state to be extracted from is declared

def extraction_func(state, state_list)
    for s_list in state_list:
        if s_list[0] == state:
        #this condition checks for the exaxt state you want to extract from.
            LGAlist = s_list[1] #assigns the list of LGAs of that particular state to a variable for further looping
            for l_list in LGAlist:
                LGA = l_list[0] #assigns the current LGA name to a variable to be used by our selenium function
                Wards = l_list[1] #assigns the list of wards of that particular LGA to a variable for further looping
                for w_list in Wards:
                    ward = w_list[0] #assigns the current ward name to a variable to be used by our selenium function
                    pollingunits = w_list[1] #assigns the list of polls of that particular ward to a variable for further looping
                    for polls in pollingunits:
                        #print("{}:{}:{}:{}".format(state,LGA,ward,polls))
                        #insert the function to webscrape the webpages
                        inec_webscrap(state_opt =state,lga_opt ='BONNY',ward_opt = 'KALAIBIAMA',polling_opt = 'VILLAGE SQUARE, OGBOKUKIRI')

In [ ]:
#in the event that the extraction process is broken
state = 'lagos' #state in our state_list to be cleaned is declared
captchabrokenafter = 'punit1.1' #the last polling unit to be extracted, after which our code broke due to captcha is 

def state_listcleaner(state, captchabrokenafter, state_list):
    #only use when the code has been broken by the captcha
    for s_list in state_list:
        if s_list[0] == state:
        #check to for the state to clean the already extracted polling units from.
            LGAlist = s_list[1] #assigns the list of LGAs of that particular state to a variable for further looping
            for l_list in LGAlist:
                Wards = l_list[1] #assigns the list of wards of that particular LGA to a variable for further looping
                for w_list in Wards:
                    pollingunits = w_list[1] #assigns the list of polls of that particular ward to a variable for further looping
                    for polls in pollingunits:
                        if polls == captchabrokenafter: 
                        #checks for the index of the polling unit last extracted, and remove all before it as they must have been extracted.
                            index = pollingunits.index(polls)
                            del pollingunits[:index+1]
                            break
    return (state_list)